In [1]:
import pandas as pd
import QuantLib as ql

# Description
This notebook builds EUR-collateralized FX forward and xccy quotes given that we know:
- EUR discounting curve (based on €STR OIS rates),
- USD discounting curve (based on SOFR OIS rates),
- USD-collateralized-in-EUR curve (henceforth _modified USD curve_),
- spot FX rate.

In [2]:
date = ql.Date(18,8,2022)
ql.Settings.instance().evaluationDate = date

eur_rate = 0.01
usd_rate = 0.015
usd_rate_mod = 0.018
spot_fx = 1.2 # EURUSD (base currency = EUR)

expiries_fwd  = ['1W','1M', '3M', '6M', '9M', '12M']
maturities_xccy = ['2Y', '5Y', '10Y', '15Y', '20Y', '30Y']

In [3]:
eur_curve     = ql.YieldTermStructureHandle(ql.FlatForward(date, ql.QuoteHandle(ql.SimpleQuote(eur_rate)),     ql.SimpleDayCounter()))
usd_curve     = ql.YieldTermStructureHandle(ql.FlatForward(date, ql.QuoteHandle(ql.SimpleQuote(usd_rate)),     ql.SimpleDayCounter()))
usd_curve_mod = ql.YieldTermStructureHandle(ql.FlatForward(date, ql.QuoteHandle(ql.SimpleQuote(usd_rate_mod)), ql.SimpleDayCounter()))

# Build FX forward quotes

In [18]:
# build FX forward quotes from EUR and mod USD rates
# FWD_EURUSD = SPOT_EURUSD * dEUR/dmodUSD
quotes_fx = []
for expiry in expiries_fwd:
    fwd_maturity = ql.NullCalendar().advance(date, ql.Period(expiry))
    quotes_fx.append(spot_fx * eur_curve.discount(fwd_maturity)/usd_curve_mod.discount(fwd_maturity))

quotes_fx = pd.DataFrame({'expiry': expiries_fwd, 'fx_fwd':quotes_fx})
quotes_fx
    


,expiry,fx_fwd
0,1W,1.200187
1,1M,1.200800
2,3M,1.202402
3,6M,1.204810
4,9M,1.207222
5,12M,1.209639


# Build xccy quotes
To build the xccy quotes from the curves, we create a helper `xccy_swap` object, into which we pass inputs.

Then we search for such a fair spread on the EUR leg that renders (€ESTR + EUR spread) vs SOFR xccy fair, if we discount by €STR curve EUR cash flow and modified USD curve the USD cash-flow. 

In [5]:
class xccy_swap:

    def __init__(self, pricing_date: ql.Date, eur_fcst_curve, usd_fcst_curve, tenor_years, spread_eur = None):

        self.inputs = locals()
        self.inputs.pop('self');

        self.pricing_date = pricing_date
        self.schedule    = ql.MakeSchedule(pricing_date, ql.NullCalendar().advance(pricing_date, ql.Period(tenor_years, ql.Years)), ql.Period('1Y'))

        if spread_eur:
            self.build_swap(pricing_date, spread_eur, eur_fcst_curve, usd_fcst_curve, tenor_years)

        

    def build_swap(self, pricing_date: ql.Date, spread_eur, eur_fcst_curve, usd_fcst_curve, tenor_years):
        
        eur_leg     = ql.OvernightLeg([1], self.schedule, ql.Eonia(eur_fcst_curve),ql.SimpleDayCounter(), ql.Following, [1], [spread_eur], True)
        usd_leg     = ql.OvernightLeg([1], self.schedule, ql.Sofr(usd_fcst_curve), ql.SimpleDayCounter(), ql.Following, [1], [0], True)
        self.eur_leg = eur_leg
        self.usd_leg = usd_leg

        self.eur_notional = ql.Leg([ql.SimpleCashFlow(1.0, max(self.schedule))])
        self.usd_notional = ql.Leg([ql.SimpleCashFlow(1.0, max(self.schedule))])

    def npv(self, eur_discount_curve, usd_discount_curve):
        eur_npv = ql.CashFlows.npv(self.eur_leg, eur_discount_curve, True, self.pricing_date) + ql.CashFlows.npv(self.eur_notional, eur_discount_curve, True, self.pricing_date)
        usd_npv = ql.CashFlows.npv(self.usd_leg, usd_discount_curve, True, self.pricing_date) + ql.CashFlows.npv(self.usd_notional, usd_discount_curve, True, self.pricing_date)

        return eur_npv - usd_npv

    def fair_spread(self, eur_discount_curve, usd_discount_curve):
        from scipy.optimize import root_scalar
        inputs = self.inputs
        inputs.pop('spread_eur');

        return root_scalar(lambda spread: xccy_swap(**inputs, spread_eur=spread).npv(eur_discount_curve, usd_discount_curve), x0 = -0.1 ,x1 = 0.1).root

In [19]:
# iterate over all xccy maturities and find xccy spreads
# the xccy exchange: €STR + EUR_spread vs SOFR
 
xccy_spreads_eur = []
for maturity in maturities_xccy:
    tenor_years = ql.Period(maturity).length()
    xccy_spreads_eur.append(xccy_swap(date, eur_curve, usd_curve, tenor_years).fair_spread(eur_curve, usd_curve_mod))

xccy_spreads_eur = pd.DataFrame({'expiry': maturities_xccy, 'EUR_spread': xccy_spreads_eur})
xccy_spreads_eur

,expiry,EUR_spread
0,2Y,-0.003014
1,5Y,-0.002981
2,10Y,-0.002926
3,15Y,-0.002871
4,20Y,-0.002819
5,30Y,-0.002721
